<a href="https://colab.research.google.com/github/neutrino-dot/vocal/blob/main/%E6%AF%8D%E9%9F%B3%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 125Hzのpulse
import numpy as np
from scipy.signal import square
from IPython.display import Audio

fs = 44100
duration = 1.5
t = np.linspace(0, duration, int(fs*duration), endpoint=False)

f = 125  # 周波数[Hz]
base = square(2 * np.pi * f * t, duty=0.5)

Audio(base, rate=fs, autoplay=True)

In [17]:
#@title あいうえお
import numpy as np
from scipy.signal import square, butter, sosfilt
from IPython.display import Audio


# 設定
fs = 44100 # サンプリング周波数(Hz)
duration = 1 # 各声の長さ(s)


# 基本pulse振動
t = np.linspace(0, duration * 5, int(fs * duration *5), endpoint=False)
f0 = 125 # 基本周波数(Hz)
base_pulse =  square(2 * np.pi * f0 * t,duty=0.125) # 基本振動(dutyは12.5%)


# vocal定義
total_samples = int(fs * 6) # 全サンプル数
vocal = np.zeros(total_samples) # 声を入れる配列


formants = []

# あ母音のフォルマント (F1, F2, F3)
formants += [[
    (800, 150),
    (1200, 200),
    (2600, 300)
]]

# い母音のフォルマント (F1, F2, F3)
formants += [[
    (300, 150),
    (2300, 200),
    (2900, 300)
]]

# う母音のフォルマント (F1, F2, F3)
formants += [[
    (300, 150),
    (1200, 200),
    (2500, 300)
]]

# え母音のフォルマント (F1, F2, F3)
formants += [[
    (500, 150),
    (1900, 200),
    (2500, 300)
]]

# お母音のフォルマント (F1, F2, F3)
formants += [[
    (500, 150),
    (800, 200),
    (2500, 300)
]]


# 各フォルマントのBPFを適用して足し合わせる
def bpf(wave,i):
  filtered = np.zeros_like(wave)
  for fc, bw in formants[i]:
      low = fc - bw / 2
      high = fc + bw / 2
      sos = butter(2, [low, high], btype='band', fs=fs, output='sos')
      filtered += sosfilt(sos, wave)
  return filtered


# 各声を追加
for i in range(5):
  start_sample = int(fs * i)
  end_sample = int(fs * (i + duration))
  vocal[start_sample:end_sample] += bpf(base_pulse[start_sample:end_sample],i)


# 正規化
vocal /= np.max(np.abs(vocal))


# 再生
Audio(vocal, rate=fs, autoplay=True)

### **参考サイト:**
[「人の声」のつくりかた：合成音声の仕組み](https://smartsoundlab.com/2019/01/000047.html)